In [1]:
import nagisa
import requests
from bs4 import BeautifulSoup

[dynet] random seed: 1234
[dynet] allocating memory: 32MB
[dynet] memory allocation done.


In [2]:
URL = "https://www.aozora.gr.jp/cards/000148/files/795_43522.html"
book_source = requests.get(URL)

In [6]:
soup = BeautifulSoup(book_source.content, "html.parser")
book_text = soup.find("div",{"class":"main_text"}).get_text()

In [7]:
verbs = nagisa.extract(book_text, extract_postags='動詞')
nouns = nagisa.extract(book_text, extract_postags='名詞')
adjectives = nagisa.extract(book_text, extract_postags='形容詞')

In [8]:
print(verbs.words)

['描い', '持っ', 'いる', '思っ', '入れ', 'しまっ', 'おい', '経つ', '伴れ', '忘れ', '打ち', '過ぎる', '思い出し', 'し', 'おい', 'する', '知れ', 'やっ', '仕立て', '云う', '起っ', '引き出し', '払い', '検べる', '畳ん', 'あっ', '思っ', '出', '来', '宛て', 'し', '分ら', '選び出し', '挟ん', 'さ', '挿し', 'し', 'ある', '思わ', '突い', '描い', 'いう', '加え', 'ある', 'もっ', '見る', '考え', 'い', '描い', 'い', '活け', '帰り来る', '云う', '添え', '送っ', '来', 'ある', '懸け', '眺め', '見る', 'する', '合せ', '使っ', '開い', 'ある', 'し', '見', 'あっ', '眺め', '襲っ', '来', 'なら', '描く', '惜しま', '見える', 'かけ', '塗り上げ', 'いる', 'し', '要する', '作り上げる', '云っ', 'ある', '思う', '云う', 'おけ', '聞い', 'しよう', '企て', 'し', '向っ', 'する', '忘れ', 'する', 'あろう', 'よっ', 'さ', 'ある', '呵し', 'なす', '浸る', 'なっ', '竦ん', 'しまっ', '思う', '禁じ', '得', 'ある', '来', '見', 'あり', '云っ', 'ある', '出す', '費さ', 'なら', '思う', 'いら', 'さ', 'ある', '隠し', '切れ', '溢れ', 'いる', '思う', '答え', '利い', 'あり', 'ある', 'すれ', 'ある', 'つけ', '弁ずる', 'やむ', 'え', '棄て', 'し', 'すれ', 'し', '免れ', 'し', 'し', 'し', 'あっ', 'し', '笑い', '得る', '捉え', '得', '惚れ込ん', 'なろう', 'する', '描い', '認める', 'でき', 'し', 'せ', 'せ', 'とっ', 'ある', 'でき', '得る', 'さ', 'し']


In [9]:
requests.get("https://jisho.org/api/v1/search/words?keyword=読み").json()

{'meta': {'status': 200},
 'data': [{'slug': '読み',
   'is_common': True,
   'tags': [],
   'jlpt': ['jlpt-n3'],
   'japanese': [{'word': '読み', 'reading': 'よみ'}],
   'senses': [{'english_definitions': ['reading'],
     'parts_of_speech': ['Noun'],
     'links': [],
     'tags': [],
     'restrictions': [],
     'see_also': [],
     'antonyms': [],
     'source': [],
     'info': []},
    {'english_definitions': ['reading (of a kanji, esp. kun reading)'],
     'parts_of_speech': ['Noun'],
     'links': [],
     'tags': [],
     'restrictions': [],
     'see_also': ['訓 くん'],
     'antonyms': [],
     'source': [],
     'info': ['also written 訓み']},
    {'english_definitions': ['reading (e.g. of a situation)',
      'insight',
      'judgment',
      'interpretation',
      'prediction',
      'calculation'],
     'parts_of_speech': ['Noun'],
     'links': [],
     'tags': [],
     'restrictions': [],
     'see_also': [],
     'antonyms': [],
     'source': [],
     'info': []}],
   'attri